# Testes

### Imports

In [ ]:
from os.path import join, exists
from os import makedirs
from json import loads, dumps

from unsloth import FastVisionModel
from tqdm.notebook import tqdm
from PIL import Image

import torch

from scripts.authentication import authenticate_huggingface
from scripts.messages import add_inference_message, format_prompt

import scripts.configuration as config

### Configurações

In [ ]:
# Edite as duas constantes abaixo
MODEL = config.BASE_MODEL_NAME
QUANTIZED = False  # Isso é sobreescrito no caso de modelos treinados
TEST_ON_TRAINING = False  # Testa o modelo sobre os dados de treinamento

with open(join(config.RESULTS_PATH, 'models.json'), 'r', encoding='utf-8') as file:
    models = loads(file.read())

model_stats = models[MODEL]
model_path = ''

if model_stats['local']:
    model_path = join(config.RESULTS_PATH, 'adapter_weights', MODEL)
else:
    model_path = MODEL

quantized = model_stats['quantized'] if model_stats['quantized'] is not None else QUANTIZED
prompt_template = ''
prompt_type = None

match model_stats['type']:
    case 'base' | 'simple_classification':
        prompt_type = config.PromptType.SIMPLE_CLASSIFICATION
        prompt_template = config.SIMPLE_CLASSIFICATION_PROMPT_TEMPLATE
    case 'full_classification':
        prompt_type = config.PromptType.FULL_CLASSIFICATION
        prompt_template = config.FULL_CLASSIFICATION_PROMPT_TEMPLATE
    case 'report':
        prompt_type = config.PromptType.REPORT
        prompt_template = config.REPORT_PROMPT_TEMPLATE
    case _:
        raise ValueError('Invalid model type')

model_version = model_stats['version']
model_size = model_stats['size']

### Autenticação

In [ ]:
authenticate_huggingface()

### Carregamento do dataset

In [ ]:
if TEST_ON_TRAINING:
    with open(join(config.DATA_PATH, 'stt_data', 'training_dataset.json'), 'r', encoding='utf-8') as file:
        training_dataset = loads(file.read())

    with open(join(config.DATA_PATH, 'training_dataset_analysis.json'), 'r', encoding='utf-8') as file:
        training_dataset_analysis = loads(file.read())

with open(join(config.DATA_PATH, 'stt_data', 'test_dataset.json'), 'r', encoding='utf-8') as file:
    test_dataset = loads(file.read())

with open(join(config.DATA_PATH, 'test_dataset_analysis.json'), 'r', encoding='utf-8') as file:
    test_dataset_analysis = loads(file.read())

### Carregamento do modelo

In [ ]:
model, tokenizer = FastVisionModel.from_pretrained(
    model_path,
    load_in_4bit=quantized,
    use_gradient_checkpointing='unsloth',
)

FastVisionModel.for_inference(model)

### Preparação do teste

In [ ]:
formatted_prompt_on_test = format_prompt(prompt_template, prompt_type, test_dataset_analysis)
messages_on_test = add_inference_message(formatted_prompt_on_test)

# Idealmente isso seria igual ao de teste
if TEST_ON_TRAINING:
    formatted_prompt_on_training = format_prompt(prompt_template, prompt_type, training_dataset_analysis)
    messages_on_training = add_inference_message(formatted_prompt_on_training)

test_name = f'{MODEL}_test'.strip('unsloth/')
test_output = {'model': MODEL, 'results_on_test': [], 'results_on_training': []}
tests_path = join(config.RESULTS_PATH, 'tests')

if not exists(tests_path):
    makedirs(tests_path)

### Testes sobre os dados de teste

In [ ]:
input_text = tokenizer.apply_chat_template(messages_on_test, add_generation_prompt=True)

for idx, exam in enumerate(tqdm(test_dataset, desc='Testing on test data: ')):
    for image_name in exam['images']:
        image_path = join(config.DATA_PATH, 'stt_raw_data', 'dataset', 'images', image_name)
        image = Image.open(image_path).convert('RGB')

        inputs = tokenizer(
            image,
            input_text,
            add_special_tokens=False,
            return_tensors='pt',
        ).to('cuda')

        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            use_cache=True,
            temperature=0.01,
            min_p=0.1
        )

        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        assistant_message = result.split('assistant')[-1].strip()
        test_output['results_on_test'].append({'exam_id': exam['id'], 'answer': assistant_message})

### Testes sobre os dados de treinamento

In [ ]:
if TEST_ON_TRAINING:
    input_text = tokenizer.apply_chat_template(messages_on_test, add_generation_prompt=True)

    for idx, exam in enumerate(tqdm(training_dataset, desc='Testing on training data: ')):
        for image_name in exam['images']:
            image_path = join(config.DATA_PATH, 'stt_raw_data', 'dataset', 'images', image_name)
            image = Image.open(image_path).convert('RGB')

            inputs = tokenizer(
                image,
                input_text,
                add_special_tokens=False,
                return_tensors='pt',
            ).to('cuda')

            outputs = model.generate(
                **inputs,
                max_new_tokens=2048,
                use_cache=True,
                temperature=0.01,
                min_p=0.1
            )

            result = tokenizer.decode(outputs[0], skip_special_tokens=True)
            assistant_message = result.split('assistant')[-1].strip()
            test_output['results_on_training'].append({'exam_id': exam['id'], 'answer': assistant_message})

### Salvamento do teste

In [ ]:
test_path = join(tests_path, f'{test_name}.json')

with open(test_path, 'w+', encoding='utf-8') as file:
    file.write(dumps(test_output, indent=4, ensure_ascii=False))